In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('smsspamcollection.tsv',sep='\t')

In [7]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [8]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [9]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
##using text as a feature
X = df['message']

In [12]:
y = df['label']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
count_vect = CountVectorizer()

In [16]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [17]:
#X is still in this form, we have to vectorize it
#way 1
#first vectorizer to data, (build a vocab, count no of words)
#trnasform original mssg to vector

In [18]:
#count_vect.fir(X_train)  #build vocab
#X_train_counts = count_vect.transform(X_train) #transform

#or easy, combining above two commands
X_train_counts = count_vect.fit_transform(X_train)

#X-train_counts is a sparce matrix

In [19]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [20]:
X_train.shape

(3733,)

In [21]:
###across mssgs, 7082 unique words


In [22]:
X_train_counts.shape

(3733, 7082)

In [23]:
#TF_IDF
# trem freq inverse doc freq
from sklearn.feature_extraction.text import TfidfTransformer

In [24]:
#create instance
tfidf_transformer = TfidfTransformer()

In [25]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [26]:
X_train_tfidf.shape

(3733, 7082)

In [27]:
# combining above steps of count vectorization and tfidf transformation
# alternative:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

In [28]:
from sklearn.svm import LinearSVC

In [29]:
clf = LinearSVC()

In [30]:
clf.fit(X_train_tfidf,y_train)  ##will display model used

LinearSVC()

In [31]:
#pipeline class can be used to vectorize and classification
# everything above 

from sklearn.pipeline import Pipeline

#this pipeline doing vectorization and SVC, support vector classifier
text_clf = Pipeline([('tfidf',TfidfVectorizer()), ('clf',LinearSVC())])

In [33]:
## what we did till now:
"""
call vectorizer
create instance of vectorizer
fit tranform on training dataset
import classifier (SVC)
fit vectorized version of dataset
"""

#pipeline does all that in single command
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [34]:
predictions = text_clf.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix, classification_report

In [36]:
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [38]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [41]:
from sklearn import metrics
metrics.accuracy_score(y_test,predictions)

0.989668297988037

In [42]:
## predicting on new mssg

In [44]:
text_clf.predict(["hello shantanu"])

array(['ham'], dtype=object)

In [45]:
text_clf.predict(["congratulations you won money"])

array(['spam'], dtype=object)